In [1]:
import pandas as pd
import pyspark
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pyspark.sql import SparkSession
import pyspark.sql.types as t
import pyspark.sql.functions as f
from sklearn.covariance import LedoitWolf
from scipy.stats import rankdata

In [2]:
sc = SparkSession.builder.getOrCreate()

In [3]:
full_data = pd.read_csv("full_data_df")
baseline = pd.read_csv("baseline_df")

In [4]:
full_data.shape
baseline.shape

(5000, 8)

In [6]:
df = full_data[['A','B','C','D','label']].sample(n=15000)
sdf = sc.createDataFrame(df)
bdf = baseline[['A','B','C','D','label']].sample(n=5000)
baseline_df = sc.createDataFrame(bdf)

### This code defines a udf that finds average mahalanobis depth

In [7]:
@f.udf(t.FloatType())
def avg_mahal(*args):
    num_rows = len(args[0])
    
    if num_rows >= 10:
        
        df = np.array(args).T
        samples = df
            
        centroid = np.average(df, axis = 0)
        shrunk = LedoitWolf()
        s_i = shrunk.fit(df).precision_
        depths = []

        for row in samples:    
            cent_dist = row - centroid
            mahal = 1/(1 + (cent_dist @ s_i @ cent_dist.T))
            depths.append(mahal)

        return float(np.average(depths))
    
    else:
        return None

In [16]:
result = sdf.groupBy("label").agg(avg_mahal(
    f.collect_list("A"),
    f.collect_list("B"),
    f.collect_list("C"),
    f.collect_list("D"),
)).sort('label')

In [17]:
result.show()

+-----+-----------------------------------------------------------------------------------------------------+
|label|avg_mahal(collect_list(A, 0, 0), collect_list(B, 0, 0), collect_list(C, 0, 0), collect_list(D, 0, 0))|
+-----+-----------------------------------------------------------------------------------------------------+
|    0|                                                                                            0.7666802|
|    1|                                                                                            0.5608252|
|    2|                                                                                            0.6621632|
|    3|                                                                                            0.5428911|
|    4|                                                                                           0.28688452|
|    5|                                                                                           0.50173885|
|    6|   

In [18]:
def simple_depth(samples, df):
    #input df of samples and distribution to base depth on
    #samples can either be a dataframe or array of n dimensional data points
    #or a single data point
    
    df = np.array(df).T
    samples = np.array(samples).T
    centroid = np.average(df, axis = 0)
    
    depths = []
    
    for row in samples:    
        cent_dist = row - centroid
        length = 1/(1+np.sqrt(np.sum(np.square(cent_dist))))
        depths.append(length)
        
    return depths


In [19]:

def mahal_shrinkage(samples, df):
    """input df of samples and distribution to base depth on samples can either be a
    dataframe or array of n dimensional data points, or a single data point
    The ledoit wolf sshrinkage operation transforms the covariance matrix in a way that 
    makes finding the inverse easier.
    """
    df = np.array(df).T
    samples = np.array(samples).T
    
    centroid = np.average(df, axis = 0)
    shrunk = LedoitWolf()
    s_i = shrunk.fit(df).precision_
    depths = []
    
    for row in samples:    
        cent_dist = row - centroid
        mahal = 1/(1 + (cent_dist @ s_i @ cent_dist.T))
        depths.append(mahal)

    return depths


In [20]:
def mkw(sample1, sample2):
    # statistic indicating if the samples are similarly distributed or not
    sample1 = np.array(sample1)
    sample2 = np.array(sample2)
    n1 = sample1.shape[0]
    n2 = sample2.shape[0]
    n = n1+n2
    
    s1_depths = [mahal_shrinkage(sample,sample1)for sample in (sample1,sample2)]
    s2_depths = [mahal_shrinkage(sample,sample2)for sample in (sample1,sample2)]
    
    s1_ranks = rankdata((s1_depths[0]+s1_depths[1]))
    s2_ranks = rankdata((s2_depths[0]+s2_depths[1]))
    
    ranks_mat = np.array([s1_ranks[:n1], s1_ranks[n1:], s2_ranks[:n2], s2_ranks[n2:]])
    sum_sq_avg =[np.square(np.sum(ranks))/ranks.shape[0] for ranks in ranks_mat]
    
    
                       
    h_stat_1 = 6/(n*(n+1))
    h_stat_2 = np.sum(sum_sq_avg)
    h_stat_3 = 3*(n+1)
        
    return h_stat_1*h_stat_2-h_stat_3

In [21]:
base=baseline[['A','B','C','D']].T
@f.udf(t.FloatType())
def mkw_baseline(*args):
    num_rows = len(args[0])
    
    if num_rows >= 10:
        
        statistic = mkw(args, base)
        
        return float(statistic)
    else:
        return None

In [22]:
result = sdf.groupBy("label").agg(mkw_baseline(
    f.collect_list("A"),
    f.collect_list("B"),
    f.collect_list("C"),
    f.collect_list("D"),
)).sort('label')

In [ ]:
result.show()

+-----+--------------------------------------------------------------------------------------------------------+
|label|mkw_baseline(collect_list(A, 0, 0), collect_list(B, 0, 0), collect_list(C, 0, 0), collect_list(D, 0, 0))|
+-----+--------------------------------------------------------------------------------------------------------+
|    0|                                                                                            1.5118593E10|
|    1|                                                                                            1.2982785E10|
|    2|                                                                                           1.00770857E10|
|    3|                                                                                             8.8240896E9|
|    4|                                                                                             6.6817874E9|
|    5|                                                                                         